# Read tables

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from reddit.reddit_streaming import *
import datetime as dt
import pprint
pp = pprint.PrettyPrinter(indent = 1)
%load_ext sparksql_magic

creds, config = read_files()
subreddit_list = config["subreddit"]
kafka_host = config["kafka_host"]
spark_host = config["spark_host"]
aws_client = creds["aws-client"]
aws_secret = creds["aws-secret"]

for subreddit in subreddit_list:
    spark = SparkSession.builder.appName("reddit_" + subreddit + "_read_data") \
                        .master("spark://{}:7077".format(spark_host)) \
                        .config("spark.scheduler.mode", "FAIR") \
                        .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
                        .config("spark.executor.memory", "1024m") \
                        .config("spark.executor.cores", "2") \
                        .config("spark.streaming.concurrentJobs", "4") \
                        .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
                        .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
                        .config("spark.sql.debug.maxToStringFields", 1000) \
                        .config("spark.eventLog.enabled", "true") \
                        .config("spark.eventLog.dir", "file:///opt/workspace/events") \
                        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-core_2.12:1.2.1") \
                        .config("spark.hadoop.fs.s3a.access.key", aws_client) \
                        .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
                        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                        .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
                        .enableHiveSupport() \
                        .getOrCreate()

    print("created spark successfully.")

    df = spark.read.format("delta").option("header", True).load("s3a://reddit-stevenhurwitt/" + subreddit + "_clean/")
    
    df.createOrReplaceTempView("reddit_{}".format(subreddit))
    print("created table reddit_{}.".format(subreddit))


:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eaa00a86-932d-4cf4-8b10-c74c4eaa5b3d;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.

created spark successfully.


22/06/17 18:47:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


created table reddit_technology.
created spark successfully.


created table reddit_ProgrammerHumor.
created spark successfully.


## technology

In [2]:
%%sparksql

select CAST(created_utc AS DATE), count(*) from reddit_technology 
group by CAST(created_utc AS DATE) 
order by CAST(created_utc AS DATE) asc

created_utc,count(1)
2022-05-11,3
2022-05-12,33
2022-05-13,27
2022-05-16,15
2022-06-14,44
2022-06-15,42
2022-06-16,42
2022-06-17,5


In [3]:
%%sparksql

select * from reddit_technology order by created_utc asc

only showing top 20 row(s)


approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,post_hint,content_categories,is_self,subreddit_type,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
null,technology,,t2_8pgkdmnc,False,null,0,False,Tech is hitting the brakes on hiring even as other industries keep adding jobs,r/technology,False,6,general,0,93,null,True,t3_unm15z,False,dark,1.0,null,3,0,140,null,False,null,False,False,null,Business,False,3,null,False,False,https://a.thumbs.redditmedia.com/0YC2GoTdt-YfMeBaTlYL2FNrrNVCUUSfaPFXbgnN4Q4.jpg,False,null,link,null,False,public,2022-05-11 22:38:56,text,6,null,null,text,cnbc.com,False,null,null,null,null,https://www.cnbc.com/amp/2022/05/11/tech-layoffs-hiring-slowdown-stand-out-in-red-hot-job-market.html,null,False,False,True,False,False,False,49cac61c-a816-11e9-be34-0ebbab5890a0,True,False,False,null,False,null,null,null,t5_2qh16,False,null,null,null,,unm15z,True,null,saurin212,null,1,True,all_ads,False,False,null,/r/technology/comments/unm15z/tech_is_hitting_the_brakes_on_hiring_even_as/,all_ads,False,https://www.cnbc.com/amp/2022/05/11/tech-layoffs-hiring-slowdown-stand-out-in-red-hot-job-market.html,11961363,2022-05-11 22:38:56,0,null,False
null,technology,,t2_50rzo,False,null,0,False,"Appeals court lifts stay, allows TX social media law to go into effect",r/technology,False,6,general,0,70,null,True,t3_unm8vy,False,dark,1.0,null,3,0,140,null,False,null,False,False,null,Politics,False,3,null,False,False,https://b.thumbs.redditmedia.com/nQHl8WXhGBisDJXlmDGVXpNRldbpWMH1xldfflbxWAc.jpg,False,null,link,null,False,public,2022-05-11 22:49:36,text,6,null,null,text,protocol.com,False,null,null,null,null,https://www.protocol.com/bulletins/texas-bias-law-proceeds,null,False,True,True,False,False,False,70f05306-a816-11e9-b261-0e41843d240e,True,False,False,null,False,null,null,null,t5_2qh16,False,null,null,null,,unm8vy,True,null,macnbc,null,0,True,all_ads,False,False,null,/r/technology/comments/unm8vy/appeals_court_lifts_stay_allows_tx_social_media/,all_ads,False,https://www.protocol.com/bulletins/texas-bias-law-proceeds,11961400,2022-05-11 22:49:36,0,null,False
null,technology,,t2_5w9r3,False,null,0,False,Court lets Texas restrictions on social platform content moderation take effect,r/technology,False,6,general,0,73,null,True,t3_unmsdt,False,dark,1.0,null,2,0,140,null,False,null,False,False,null,Social Media,False,2,null,False,True,https://b.thumbs.redditmedia.com/ncGR-bgoKIUbuE-gMQY2lAODE5fyDOww-ZF78xEBk0s.jpg,False,null,link,null,False,public,2022-05-11 23:17:20,text,6,null,null,text,theverge.com,False,null,null,null,null,https://www.theverge.com/2022/5/11/23067002/texas-netchoice-paxton-hb20-social-media-law-fifth-circuit-appeals-court-grant-stay-ruling,null,False,False,True,False,False,False,7d4d8376-a816-11e9-a92d-0e6b9fa95170,True,False,False,null,False,null,null,nul

## news

In [4]:
%%sparksql

select CAST(created_utc AS DATE), count(*) from reddit_news
group by CAST(created_utc AS DATE) 
order by CAST(created_utc AS DATE) asc

created_utc,count(1)
2022-06-17,4


In [5]:
%%sparksql

select * from reddit_news order by created_utc asc

approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,post_hint,content_categories,is_self,subreddit_type,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
null,news,,t2_bs1la4xg,False,null,0,False,"Mortgage rates hit 5.78%, the biggest weekly jump since 1987",r/news,False,6,null,0,null,null,True,t3_ved5yu,False,dark,0.9599999785423279,null,54,0,null,null,False,null,False,False,null,null,False,54,null,False,False,,False,null,null,null,False,public,2022-06-17 13:26:24,text,6,null,null,text,cnn.com,False,null,null,null,null,https://www.cnn.com/2022/06/16/homes/mortgage-rates-june-16/index.html,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh3l,False,null,null,null,,ved5yu,True,null,Dizzy-Noise-583,null,16,True,all_ads,False,False,null,/r/news/comments/ved5yu/mortgage_rates_hit_578_the_biggest_weekly_jump/,all_ads,False,https://www.cnn.com/2022/06/16/homes/mortgage-rates-june-16/index.html,24820150,2022-06-17 13:26:24,0,null,False
null,news,,t2_tlqk8,False,null,0,False,"West Memphis police investigating Friday shooting incident, 2-year-old dies from injuries",r/news,False,6,null,0,null,null,True,t3_vee3iw,False,dark,1.0,null,4,0,null,null,False,null,False,False,null,null,False,4,null,False,False,,False,null,null,null,False,public,2022-06-17 14:11:12,text,6,null,null,text,katv.com,False,null,null,null,null,https://katv.com/news/local/west-memphis-police-investigating-friday-shooting-incident-2-year-old-dies-from-injury-baptist-memorial-hospital-crittenden-crittenden-county-crime-stoppers-crime-scene-interview-witnesses-arkansas-news,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh3l,False,null,null,null,,vee3iw,True,null,Dillatrack,null,0,True,all_ads,False,False,null,/r/news/comments/vee3iw/west_memphis_police_investigating_friday_shooting/,all_ads,False,https://katv.com/news/local/west-memphis-police-investigating-friday-shooting-incident-2-year-old-dies-from-injury-baptist-memorial-hospital-crittenden-crittenden-county-crime-stoppers-crime-scene-interview-witnesses-arkansas-news,24820176,2022-06-17 14:11:12,0,null,False
null,news,,t2_3qwsyobo,False,null,0,False,‘Unacceptable’: Russian warship accused of violating Danish waters,r/news,False,6,null,0,null,null,True,t3_veebzd,False,dark,1.0,null,3,0,null,null,False,null,False,False,null,null,False,3,null,False,False,,False,null,null,null,False,public,2022-06-17 14:21:52,text,6,null,null,text,theguardian.com,False,null,null,null,null,https://www.theguardian.com/world/2022/jun/17/unacceptable-russian-warship-accused-of-violating-danish-waters,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh3l,False,null,null,null,,veebzd,True,null,Bangemkikkoks,null,0,True,all_ads,False,False,null,/r/news/comments/

## worldnews

In [6]:
%%sparksql

select CAST(created_utc AS DATE), count(*) from reddit_worldnews
group by CAST(created_utc AS DATE) 
order by CAST(created_utc AS DATE) asc

created_utc,count(1)
2022-06-17,8


In [7]:
%%sparksql

select * from reddit_worldnews order by created_utc asc

approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,post_hint,content_categories,is_self,subreddit_type,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
null,worldnews,,t2_yl06t,False,null,0,False,European Commission recommends granting Ukraine EU candidate status,r/worldnews,False,6,russia,0,null,null,True,t3_vednyc,False,null,1.0,null,9,0,null,null,False,null,False,False,null,Russia/Ukraine,False,9,null,False,True,,False,null,null,null,False,public,2022-06-17 13:52:00,richtext,6,null,null,text,ukrinform.net,False,null,null,null,null,https://www.ukrinform.net/rubric-polytics/3508964-european-commission-recommends-granting-ukraine-eu-candidate-status.html,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh13,False,null,null,null,null,vednyc,True,null,Ekaton,null,2,True,all_ads,False,False,null,/r/worldnews/comments/vednyc/european_commission_recommends_granting_ukraine/,all_ads,False,https://www.ukrinform.net/rubric-polytics/3508964-european-commission-recommends-granting-ukraine-eu-candidate-status.html,29034211,2022-06-17 13:52:00,0,null,False
null,worldnews,,t2_bmncpj69,False,null,0,False,China blocks joint India-US proposal to blacklist Pakistani terrorist Abdul Rehman Makki under UNSC sanctions,r/worldnews,False,6,null,0,null,null,True,t3_vedzuc,False,dark,1.0,null,4,0,null,null,False,null,False,False,null,null,False,4,null,False,False,,False,null,null,null,False,public,2022-06-17 14:06:56,text,6,null,null,text,tribuneindia.com,False,null,null,null,null,https://www.tribuneindia.com/news/nation/china-blocks-joint-india-us-proposal-to-blacklist-pakistani-terrorist-abdul-rehman-makki-under-unsc-sanctions-404681,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh13,False,null,null,null,,vedzuc,True,null,Winterisbucky,null,0,True,all_ads,False,False,null,/r/worldnews/comments/vedzuc/china_blocks_joint_indiaus_proposal_to_blacklist/,all_ads,False,https://www.tribuneindia.com/news/nation/china-blocks-joint-india-us-proposal-to-blacklist-pakistani-terrorist-abdul-rehman-makki-under-unsc-sanctions-404681,29034225,2022-06-17 14:06:56,0,null,False
null,worldnews,,t2_3z7n2wc1,False,null,0,False,UK: 50 new monkeypox cases in biggest epidemic beyond Africa,r/worldnews,False,6,null,0,null,null,True,t3_vee2pe,False,dark,1.0,null,3,0,null,null,False,null,False,False,null,null,False,3,null,False,True,,False,null,null,null,False,public,2022-06-17 14:11:12,text,6,null,null,text,thehill.com,False,null,null,null,null,https://thehill.com/homenews/ap/ap-health/uk-50-new-monkeypox-cases-in-biggest-epidemic-beyond-africa/,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh13,False,null,null,null,,vee2pe,True,null,Arpith2019,null,0,True,all_ads,False,False,null,/r/world

## ProgrammerHumor

In [8]:
%%sparksql

select CAST(created_utc AS DATE), count(*) from reddit_ProgrammerHumor
group by CAST(created_utc AS DATE) 
order by CAST(created_utc AS DATE) asc

created_utc,count(1)
2022-06-17,5


In [9]:
%%sparksql

select * from reddit_ProgrammerHumor order by created_utc asc

approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,post_hint,content_categories,is_self,subreddit_type,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
null,ProgrammerHumor,,t2_54dvd6d4,False,null,0,False,"Never used python, ask me a question and I’ll answer like I did.",r/ProgrammerHumor,False,6,null,0,140,null,True,t3_ved7pz,False,dark,0.800000011920929,null,3,0,140,null,False,null,True,False,null,null,False,3,null,False,False,https://a.thumbs.redditmedia.com/J5kXwF3yIEX2tqDTiAZTmQr38_lKlI8I85PPooTfu20.jpg,False,null,image,null,False,public,2022-06-17 13:30:40,text,6,null,null,text,i.redd.it,False,null,null,null,null,https://i.redd.it/i39ca2ufo6691.jpg,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2tex6,False,null,null,null,,ved7pz,True,null,TheBluePhenoix,null,5,False,all_ads,False,False,null,/r/ProgrammerHumor/comments/ved7pz/never_used_python_ask_me_a_question_and_ill/,all_ads,False,https://i.redd.it/i39ca2ufo6691.jpg,2056247,2022-06-17 13:30:40,0,null,False
null,ProgrammerHumor,,t2_75mqu1d5,False,null,0,False,The pain,r/ProgrammerHumor,False,6,meme,0,140,null,True,t3_vee1sy,False,dark,1.0,null,1,0,140,null,False,null,True,False,null,Meme,False,1,null,False,False,https://b.thumbs.redditmedia.com/vDJEMxOnIkDY7iJdkJ4v8rxt17S-3NGxeN4qO02kClw.jpg,False,null,image,null,False,public,2022-06-17 14:09:04,richtext,6,null,null,text,i.redd.it,False,null,null,null,null,https://i.redd.it/qrbzub8fv6691.jpg,null,False,True,True,False,False,False,fe277d42-440f-11e8-8ec3-0eac434afc94,True,False,False,null,False,null,null,null,t5_2tex6,False,null,null,null,,vee1sy,True,null,mcalby,null,0,True,all_ads,False,False,null,/r/ProgrammerHumor/comments/vee1sy/the_pain/,all_ads,False,https://i.redd.it/qrbzub8fv6691.jpg,2056264,2022-06-17 14:09:04,0,null,False
null,ProgrammerHumor,,t2_5wtmwnwk,False,null,0,False,Running as an admin can make it run faster...,r/ProgrammerHumor,False,6,meme,0,119,null,True,t3_veeho2,False,dark,1.0,null,5,0,140,null,False,null,True,False,null,Meme,False,5,null,False,False,https://b.thumbs.redditmedia.com/LKEWm8rZ0uyom4SOTWKWSELv9RyqYqqvK7hwp7bBMSo.jpg,False,null,image,null,False,public,2022-06-17 14:30:24,richtext,6,null,null,text,i.redd.it,False,null,null,null,null,https://i.redd.it/2bkimh0ow6691.png,null,False,False,True,False,False,False,fe277d42-440f-11e8-8ec3-0eac434afc94,True,False,False,null,False,null,null,null,t5_2tex6,False,null,null,null,,veeho2,True,null,vanessabaxton,null,1,True,all_ads,False,False,null,/r/ProgrammerHumor/comments/veeho2/running_as_an_admin_can_make_it_run_faster/,all_ads,False,https://i.redd.it/2bkimh0ow6691.png,2056353,2022-06-17 14:30:24,0,null,False
null,ProgrammerHumor,,t2_me0endh,False,null,0,False,This one feels so accurate than other pictures,r/ProgrammerHumor,False,6,null,0,140,null,T

22/06/17 18:43:41 ERROR TaskSchedulerImpl: Lost executor 2 on 172.24.0.7: worker lost
22/06/17 18:43:41 ERROR TaskSchedulerImpl: Lost executor 1 on 172.24.0.7: worker lost
22/06/17 18:43:41 ERROR TaskSchedulerImpl: Lost executor 3 on 172.24.0.7: worker lost
22/06/17 18:43:41 ERROR TaskSchedulerImpl: Lost executor 0 on 172.24.0.7: worker lost
22/06/17 18:43:42 ERROR TaskSchedulerImpl: Lost executor 5 on 172.24.0.6: worker lost
22/06/17 18:43:42 ERROR TaskSchedulerImpl: Lost executor 7 on 172.24.0.6: worker lost
22/06/17 18:43:42 ERROR TaskSchedulerImpl: Lost executor 4 on 172.24.0.6: worker lost
22/06/17 18:43:42 ERROR TaskSchedulerImpl: Lost executor 6 on 172.24.0.6: worker lost
22/06/17 18:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_49_26 !
22/06/17 18:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_66_37 !
22/06/17 18:43:43 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_15_38 !
22/06/17 18:43:43 WARN BlockMa

### stop spark

In [8]:
# spark.stop()